In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[5],3c
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,5e-25
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[5]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`              
            layer    coolrg                layer        coolrg
pressure                                                      
0.000312        1  0.000011                    1 -3.969772e-06
0.000750        2  0.000016                    2 -3.500005e-06
0.001052        3  0.000026                    3  1.768528e-06
0.001476        4  0.000038                    4  6.237878e-06
0.002070        5  0.000052                    5  1.146021e-05
0.002904        6  0.000069                    6  1.749444e-05
0.004074        7  0.000089                    7  2.492476e-05
0.005714        8  0.000112                    8  3.402778e-05
0.008015        9  0.000139                    9  4.464250e-05
0.011243       10  0.000171                   10  5.685239e-05
0.015771       11  0.000207                   11  7.089474e-05
0.022122       12  0.000249                   12  8.708560e-05
0.031031       13  0.000296                   13  1.056340e-04
0.043528       14  0.000344                   14  1.242528e-04
0.061057       15  0.000386                   15  1.403293e-04
0.085645       16  0.000424                   16  1.554952e-04
0.120136       17  0.000466                   17  1.716140e-04
0.168516       18  0.000511                   18  1.885481e-04
0.236378       19  0.000559                   19  2.062310e-04
0.331549       20  0.000611                   20  2.248373e-04
0.465100       21  0.000661                   21  2.438534e-04
0.652400       22  0.000621                   22  2.315907e-04
0.915100       23  0.000478                   23  1.785339e-04
1.283650       24  0.000341                   24  1.256788e-04
1.800600       25  0.000236                   25  8.531643e-05
2.525700       26  0.000158                   26  5.517579e-05
3.542800       27  0.000101                   27  3.294456e-05
4.969550       28  0.000064                   28  1.867240e-05
6.970850       29  0.000043                   29  1.070356e-05
9.778100       30  0.000030                   30  5.476070e-06
13.715850      31  0.000019                   31  1.299245e-06
19.239350      32  0.000012                   32 -1.864104e-06
26.987250      33  0.000009                   33 -2.808935e-06
37.855300      34  0.000012                   34 -1.517101e-06
53.100050      35  0.000015                   35  4.602666e-07
73.887500      36  0.000019                   36  2.784740e-06
97.662500      37  0.000022                   37  4.793325e-06
121.437500     38  0.000022                   38  5.551658e-06
145.212500     39  0.000029                   39  7.447509e-06
168.987500     40  0.000036                   40  9.362011e-06
192.762500     41  0.000043                   41  1.140409e-05
216.537500     42  0.000046                   42  1.263830e-05
240.312500     43  0.000048                   43  1.358035e-05
264.087500     44  0.000051                   44  1.438936e-05
287.862500     45  0.000055                   45  1.677101e-05
311.637500     46  0.000082                   46  3.558798e-05
335.412500     47  0.000160                   47  9.049646e-05
359.187500     48  0.000293                   48  1.877315e-04
382.962500     49  0.000609                   49  4.174969e-04
406.737500     50  0.001231                   50  8.886109e-04
430.512500     51  0.002219                   51  1.668828e-03
454.287500     52  0.003648                   52  2.860395e-03
478.062500     53  0.005546                   53  4.538419e-03
501.837500     54  0.007899                   54  6.730930e-03
525.612500     55  0.010643                   55  9.326038e-03
549.387500     56  0.013670                   56  1.228652e-02
573.162500     57  0.016861                   57  1.548287e-02
596.937500     58  0.020079                   58  1.875066e-02
620.712500     59  0.023207                   59  2.193507e-02
644.487500     60  0.026145                   60  2.485538e-02
668.262500     61  0.028824                  

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -18.787640  0.000000e+00 -18.787640 -18.803238   
0.000624    2     -18.787640  4.451939e-09 -18.787640 -18.803238   
0.000876    3     -18.787640  6.370098e-09 -18.787640 -18.803238   
0.001229    4     -18.787640  9.450519e-09 -18.787640 -18.803238   
0.001723    5     -18.787640  1.440286e-08 -18.787640 -18.803238   
0.002417    6     -18.787640  2.235540e-08 -18.787640 -18.803238   
0.003391    7     -18.787640  3.511338e-08 -18.787640 -18.803238   
0.004757    8     -18.787640  5.555993e-08 -18.787640 -18.803238   
0.006672    9     -18.787640  8.829336e-08 -18.787640 -18.803238   
0.009359    10    -18.787640  1.406452e-07 -18.787640 -18.803238   
0.013128    11    -18.787640  2.242893e-07 -18.787640 -18.803238   
0.018415    12    -18.787640  3.577999e-07 -18.787639 -18.803238   
0.025830    13    -18.787640  5.706941e-07 -18.787639 -18.803238   
0.036232    14    -18.787640  9.096483e-07 -18.787639 -18.803238   
0.050823    15    -18.787640  1.440999e-06 -18.787638 -18.803238   
0.071291    16    -18.787640  2.252620e-06 -18.787637 -18.803238   
0.100000    17    -18.787639  3.477501e-06 -18.787636 -18.803238   
0.140271    18    -18.787639  5.324854e-06 -18.787634 -18.803238   
0.196760    19    -18.787638  8.110076e-06 -18.787630 -18.803238   
0.275997    20    -18.787637  1.230775e-05 -18.787625 -18.803238   
0.387100    21    -18.787636  1.862926e-05 -18.787617 -18.803237   
0.543100    22    -18.787633  2.807851e-05 -18.787605 -18.803237   
0.761700    23    -18.787629  4.064750e-05 -18.787589 -18.803236   
1.068500    24    -18.787626  5.494944e-05 -18.787571 -18.803235   
1.498800    25    -18.787624  7.038117e-05 -18.787554 -18.803236   
2.102400    26    -18.787624  8.691990e-05 -18.787537 -18.803237   
2.949000    27    -18.787626  1.045842e-04 -18.787521 -18.803239   
4.136600    28    -18.787630  1.233863e-04 -18.787507 -18.803243   
5.802500    29    -18.787638  1.441208e-04 -18.787494 -18.803249   
8.139200    30    -18.787651  1.686027e-04 -18.787482 -18.803258   
11.417000   31    -18.787669  1.986091e-04 -18.787471 -18.803271   
16.014700   32    -18.787696  2.358834e-04 -18.787460 -18.803289   
22.464000   33    -18.787734  2.831573e-04 -18.787451 -18.803315   
31.510500   34    -18.787789  3.478606e-04 -18.787441 -18.803353   
44.200100   35    -18.787867  4.437352e-04 -18.787424 -18.803407   
62.000000   36    -18.787979  5.880651e-04 -18.787391 -18.803488   
85.775000   37    -18.788131  7.944337e-04 -18.787337 -18.803605   
109.550000  38    -18.788287  1.011463e-03 -18.787276 -18.803729   
133.325000  39    -18.788448  1.234831e-03 -18.787213 -18.803859   
157.100000  40    -18.788677  1.544982e-03 -18.787131 -18.804045   
180.875000  41    -18.788984  1.953287e-03 -18.787031 -18.804296   
204.650000  42    -18.789389  2.477507e-03 -18.786911 -18.804629   
228.425000  43    -18.789876  3.094841e-03 -18.786781 -18.805035   
252.200000  44    -18.790451  3.805939e-03 -18.786645 -18.805520   
275.975000  45    -18.791130  4.628218e-03 -18.786502 -18.806099   
299.750000  46    -18.791934  5.586511e-03 -18.786347 -18.806787   
323.525000  47    -18.792882  6.766246e-03 -18.786116 -18.807604   
347.300000  48    -18.794000  8.333393e-03 -18.785666 -18.808568   
371.075000  49    -18.795303  1.046351e-02 -18.784840 -18.809702   
394.850000  50    -18.797201  1.407561e-02 -18.783126 -18.811373   
418.625000  51    -18.800089  2.043051e-02 -18.779659 -18.813938   
442.400000  52    -18.804200  3.079233e-02 -18.773408 -18.817604   
466.175000  53    -18.809725  4.659048e-02 -18.763134 -18.822559   
489.950000  54    -18.816775  6.926081e-02 -18.747514 -18.828928   
513.725000  55    -18.825371  1.001038e-01 -18.725268 -18.836749   
537.500000  56    -18.835431  1.401372e-01 -18.695294 -18.845887   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')